In [13]:
import requests,json,random
# random reqId generator for post req get_approaching_veichles()
def s4():
    return ''.join([random.choice('0123456789abcdef') for x in range(4)])
def guid():
    return s4()+s4()+'-'+s4()+'-'+s4()+'-'+s4()+s4()+s4()

#req header 
headers={'content-type': 'application/json','accept':'application/json'}


class api:
    def __init__(self):
        self.baseurl="http://167.114.174.89:8080/"
        self.baseurl_tp="http://167.114.174.89:2080/"
        self.req_id=""
        self.stops=self.get_all_stoppages()["data"]
        
    #utility
    def get_stop_id_by_name(self,stop_name):
        for stop in self.stops:
            if stop["stopName"]==stop_name:
                return stop["stopId"]
        return "0"
    
    # Get req working all
    def get_all_routes(self):
        r = requests.get(self.baseurl+"app/routes/getAllRoutes.json")
        return json.loads(r.text)
        
    def get_all_stoppages(self):
        r = requests.get(self.baseurl+"app/stops/getAllStops.json")
        return json.loads(r.text)
        
    def get_last_update_time(self):
        r = requests.get(self.baseurl+"app/routes/getLastUpdateTime.json")
        return json.loads(r.text)
        
    def get_kolkata_traffic_update(self):
        r = requests.get(self.baseurl+"app/sm/kolkatatrafficupdate.json")
        return json.loads(r.text)
    
    
    # Post req . working n tested
    def get_approaching_vehicles(self,stop_id):
        data=json.dumps({'requestId':guid(),'stopId':stop_id})
        r=requests.post(self.baseurl+"app/travel/getApproachingVehicles.json",data=data,headers=headers)
        self.req_id=json.loads(r.text)['requestId']
        return json.loads(r.text)
        
    def get_vehicle_ETA(self,veichle_no):
        data=json.dumps({'vehicleNo':veichle_no})
        r=requests.post(self.baseurl+"app/eta/getVehicleETA.json",data=data,headers=headers)
        return json.loads(r.text)
        
    def get_vehicles_by_rec_boundary(self,rec):
        data=json.dumps({'pointNW':{'latitude':rec[0],'longitude':rec[1]},'pointSE':{'latitude':rec[2],'longitude':rec[3]}})
        r=requests.post(self.baseurl+"app/vehicles/getVehicleList.json",data=data,headers=headers)
        return json.loads(r.text)
        
    def get_stoppages_by_rec_boundary(self,rec):
        data=json.dumps({'pointNW':{'latitude':rec[0],'longitude':rec[1]},'pointSE':{'latitude':rec[2],'longitude':rec[3]}})
        r=requests.post(self.baseurl+"app/stops/getStopList.json",data=data,headers=headers)
        return json.loads(r.text)
    
    def find_nearest_stop(self,loc,dist):
        data=json.dumps({"location": {"latitude": loc[0],"longitude": loc[1]},"searchDistance": dist})
        r=requests.post(self.baseurl+"app/stops/getNearbyStops.json",data=data,headers=headers)
        return json.loads(r.text)
        
    def get_vehicle_pos(self,veichle_no):
        data=json.dumps({'vehicleNo':veichle_no})
        r=requests.post(self.baseurl+"app/vehicles/getVehicleByRegNo.json",data=data,headers=headers)
        return json.loads(r.text)
    
    def get_vehicle_by_route(self,route_code):
        data=json.dumps({ "routeCode": route_code })
        r=requests.post(self.baseurl+"app/vehicles/getVehicleByRoute.json",data=data,headers=headers)
        return json.loads(r.text)
        
    def get_trip_plans(self,from_stop,to_stop,hop_count):
        url_params="startStopName=" + from_stop + "&endStopName=" + to_stop + "&hopCount=" + str(hop_count)
        r=requests.get(self.baseurl_tp+"/app/tripplanner/routes?"+url_params)
        return json.loads(r.text)
    
    def get_route_path_bw_2_stops(self,route_code,from_stop_id,to_stop_id):
        data=json.dumps({"requestId":guid(),"routeCode":route_code,"startStopId":from_stop_id,"endStopId":to_stop_id})
        r=requests.post(self.baseurl+"app/paths/getPathByRoute.json",data=data,headers=headers)
        return json.loads(r.text)


In [113]:
import pymongo
myclient = pymongo.MongoClient("mongodb://192.168.0.99:27017/")
mydb = myclient["teajunction"]
mycol = mydb["TeaPath"]


In [130]:
a.get_route_path_bw_2_stops(5717,1007,23)

{u'count': 0,
 u'data': None,
 u'duration': 0,
 u'message': u'',
 u'requestId': u'c017cb75-a616-a51e-a15a509229f2',
 u'status': u'SUCCESS'}

In [122]:

import time
import datetime
from tqdm import tqdm
import json
a = api() 

list_coord = []
with open('/home/aisik/Desktop/pathadisha/ScrapPyPath/PyPath/get_bus_from_stoppages/req_stop.json') as f:
    data = json.load(f)
    store_data = data['stores']
    for data_items in store_data:
        get_coord = data_items['geometry']['coordinates']
        #print(get_coord)
        list_coord.append(get_coord)
    #print(store_data)


# To get all the data in the list which have the latitude and longitude.
print(list_coord)

whole_json = {}     # will contain the whole JSON object
# iterating over each of the coordinates
for iter_var in tqdm(range(1)): 
    t = datetime.datetime.now()
    hour=t.hour
    minutes=t.minute
    day=t.day
    for coord in list_coord:
        lat_i = float(coord[1])
        lon_i = float(coord[0])     #longitude of a coordinate
        json_timestamp = {}

        rec =  [lat_i+0.01,lon_i-0.01,lat_i-0.01,lon_i+0.01]    # to get the coordinates in 2 km radius
        get_json = a.get_vehicles_by_rec_boundary(rec)
        get_list_json = []
        for data_items in get_json['data']:
            if(data_items['journeyStarted']== True):
                data = {}
                data['properties']={}
                try:
                    data['properties']['latitude']= data_items["lastLocation"]["latitude"]
                except:
                    data['properties']['latitude'] = ""

                try:
                    data['properties']['longitude'] = data_items["lastLocation"]["longitude"]
                except:
                    data['properties']['longitude'] = ""

                try:
                    data['properties']['lastTime'] = data_items["lastTime"]
                except:
                    data['properties']["lastTime"] = ""

                try:
                    data['properties']['outOfPath'] = data_items["outOfPath"]
                except:
                    data['properties']['outOfPath'] = ""

                try:
                    data['properties']['routeCode'] = data_items["routeCode"]
                except:
                    data['properties']['routeCode'] = ""

                try:
                    data['properties']['speed'] = data_items["speed"]
                except:
                    data['properties']['speed'] = ""

                try:
                    data['properties']['stopped'] = data_items["stopped"]
                except:
                    data['properties']['stopped'] = ""

                try:
                    data['properties']['vehicleType'] = data_items["vehicle"]["vehicleType"]
                except:
                    data['properties']['vehicleType'] = ""

                try:
                    data['properties']['violatesPath'] = data_items["violatesPath"]
                except:
                    data['properties']['violatesPath'] = ""

                try:
                    data['properties']['vehicleNo'] = data_items["vehicleNo"]
                except:
                    data['properties']['vehicleNo'] = ""
                data['properties']['hour']=hour
                data['properties']['min']=minutes
                data['properties']['day']=day
                data['geometry']={}
                data['geometry']['coordinates']=[lon_i,lat_i]
                data['geometry']['type']="Point"
                get_list_json.append(data)
        if(len(get_list_json)!=0):
            x = mycol.insert_many(get_list_json)
        print lat_i,lon_i
    print iter_var   
                
    time.sleep(240)





















  0%|          | 0/1 [00:00<?, ?it/s]

[[u'88.4084675050441', u'22.58758841897607'], [u'88.35182280000004', u'22.5602948'], [u'88.4084675050441', u'22.58825902657579'], [u'88.40092580033036', u'22.49405356437818'], [u'88.36164404907379', u'22.50158536413945'], [u'88.43290160453853', u'22.56836546691676'], [u'88.41427720000002', u'22.6136803'], [u'88.35925539999994', u'22.5473972'], [u'88.35450400000002', u'22.551385'], [u'88.407786', u'22.587663'], [u'88.490015', u'22.586007'], [u'88.35438299999998', u'22.5179551'], [u'88.3469288', u'22.4774352'], [u'88.375409', u'22.563641'], [u'88.47182750000002', u'22.7031656'], [u'88.44632990000002', u'22.6520429'], [u'88.40296880000005', u'22.6063834'], [u'88.3544048', u'22.5458256'], [u'88.39457500000003', u'26.7248411'], [u'88.3935702', u'26.7256795'], [u'88.400958', u'22.577859'], [u'88.44268', u'22.623706'], [u'88.34515379999993', u'22.4944519'], [u'88.36084419999997', u'22.4735315'], [u'88.353658', u'22.624197'], [u'88.49150499999996', u'22.561926'], [u'88.459814', u'22.579522'], 











100%|██████████| 1/1 [05:35<00:00, 335.55s/it]











In [119]:
for loc in mycol.find({'store': [22.58758841897607, 88.4084675050441]}):
    print loc


{u'lastTime': 1547561768000L, u'vehicleType': u'BUS', u'hour': 19, u'min': 46, u'speed': 0.0, u'outOfPath': False, u'routeCode': u'46B:DN', u'longitude': 88.383614, u'stopped': False, u'latitude': 22.581808, u'violatesPath': u'false', u'_id': ObjectId('5c3deb328fdd0f33c83f06ef'), u'vehicleNo': u'WB07J3070', u'day': 15, u'store': [22.58758841897607, 88.4084675050441]}
{u'lastTime': 1547561753000L, u'vehicleType': u'BUS', u'hour': 19, u'min': 46, u'speed': 15.0, u'outOfPath': False, u'routeCode': u'S151:UP', u'longitude': 88.390732, u'stopped': False, u'latitude': 22.581108, u'violatesPath': u'false', u'_id': ObjectId('5c3deb328fdd0f33c83f06f0'), u'vehicleNo': u'WB23D4506', u'day': 15, u'store': [22.58758841897607, 88.4084675050441]}
{u'lastTime': 1547561764000L, u'vehicleType': u'BUS', u'hour': 19, u'min': 46, u'speed': 26.0, u'outOfPath': False, u'routeCode': u'S184:DN', u'longitude': 88.390778, u'stopped': False, u'latitude': 22.581514, u'violatesPath': u'false', u'_id': ObjectId('5c3

In [124]:
d=a.get_all_routes()

In [129]:
d['data'][1]

{u'agency': {u'agencyEmail': u'cstc@email.com',
  u'agencyId': 1,
  u'agencyName': u'CSTC',
  u'agencyPhone': u'8798089899',
  u'agencyTimezone': u'Asia/Kolkata',
  u'agencyType': u'PASSENGER_TRANSPORT',
  u'agencyUrl': u'http://cstc.org'},
 u'endStop': u'SHAPOORJI',
 u'hasPaths': True,
 u'isValid': u'Y',
 u'routeDepot': u'TARATALA',
 u'routeDepotList': [u'TARATALA'],
 u'routeId': 61,
 u'routeLongName': u'PARNASREE - SHAPOORJI',
 u'routeShortName': u'AC4A:UP',
 u'routeType': 3,
 u'skipCount': 0,
 u'startStop': u'PARNASREE'}

28


/home/aisik/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{u'n': 7717, u'ok': 1.0}